In [ ]:
import requests
import re
#получаем текст постов из инстаграмма
#получаем текст постов из инстаграмма
response = requests.get('https://graph.instagram.com/v10.0/17841400617456550/media?access_token=IGQVJXTS1uTEZAmalFGUmNuODhoaUFwdW9jc3JhVUNxdnJJMkM5R3FGN1k2UlNxeEtBTDA2Q25fNFhCN0FNQm1kbHhqbkJZAUDFjTm9rWEhITktvUFdaeWFKeVpwNk0tWGozZAW5POE5qdkpkZAE5VSzY0aQZDZD&pretty=1&fields=caption&limit=25&before=QVFIUkVEY1ViVWlOdFBuY0pZAY3VMTElvY285Tngxa2d3SXJpeU9WcVh1S1ltVmVHMUIwRjRiMVBjd2Q3N3F1amRZAdUphanZAkVEtWM3IzWlloN2xESXA0dGdR')
json_response = response.json()
file = open('data_text.txt', 'w')
for i in range(len(json_response['data'])):
  item = json_response['data'][i]['caption']
  item = re.sub(r"\@\w*", '', item)
  item = re.sub(r"\#\w*", '', item)
  item = re.sub("[A-za-z]", '', item)
  file.write(item)
while True:
  if 'next' in json_response['paging'].keys():
    new_request = json_response['paging']['next']
    response = requests.get(new_request)
    json_response = response.json()
    for i in range(len(json_response['data'])):
      if 'caption' in json_response['data'][i].keys():
        item = json_response['data'][i]['caption']
        item = re.sub(r"\@\w*", '', item)
        item = re.sub(r"\#\w*", '', item)
        item = re.sub("[A-za-z]", '', item)
        file.write(item)
  else:
    break
file.close()

In [ ]:
import pandas as pd
positiver_twit = pd.read_csv('/content/drive/MyDrive/ content/positive.csv', sep = ';', header=None)

In [ ]:
file = open('data_text.txt', 'a')
for item in positiver_twit[3]:
  item = re.sub(r"\@\w*", '', item)
  item = re.sub(r"\#\w*", '', item)
  item = re.sub("[A-za-z]", '', item)
  file.write(item)
file.close()

In [ ]:
!pip install torch==1.4.0
!pip3 install transformers==3.5.0

In [ ]:
!git clone  https://github.com/sberbank-ai/ru-gpts

fatal: destination path 'ru-gpts' already exists and is not an empty directory.


In [ ]:
!mkdir models/

mkdir: cannot create directory ‘models/’: File exists


In [ ]:
!export PYTHONPATH=${PYTHONPATH}:/ru-gpts/
!CUDA_VISIBLE_DEVICES=0 python ru-gpts/pretrain_transformers.py \
    --output_dir=models/essays \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=/content/data_text.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 3\
    --block_size 2000 \
    --overwrite_output_dir

2021-05-31 19:14:58.740964: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
05/31/2021 19:15:00 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/modeling_auto.py:837: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
05/31/2021 19:15:22 - INFO - __main__ -   Training/evaluation parameters Namespace(adam_epsilon=1e-08, block_size=2000,

In [ ]:
import numpy as np
import torch

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
tok = GPT2Tokenizer.from_pretrained("models/essays/checkpoint-3500")
model = GPT2LMHeadModel.from_pretrained("models/essays/checkpoint-3500")


In [ ]:
text = "Наше окружение определяет нашу жизнь. "
inpt = tok.encode(text, return_tensors="pt")

In [ ]:
out = model.generate(inpt, max_length=200, repetition_penalty=3.0, do_sample=True, top_k=10, top_p=1.0, temperature=10.0)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
tok.decode(out[0])

'Наше окружение определяет нашу жизнь.  :*\nСпасибо тебе :) ты просто умница!))   а вот и она:) да, кстати говоря..) в субботу с утра был концерт у одного из любимых исполнителей;-). я бы на такое мероприятие сходила:С днем Рождения!! Пусть будет хорошо!) И побольше счастья во всех Ваших мечтах=РМ)))    …:///.7 А еще сегодня были отличные выходные ) В пятницу буду опять же там отдыхать,наверное не успеваю посмотреть как дела проходят за день.В понедельник снова к любименьким )))Ахахах ну ничего особенного....не то чтоб мы прям очень старались...)У кого какой любимый праздник? Я хочу новый телефон который мне понравился по фотканам ниже или ещё более красивый? ;))))     "Полярность". Да здравствуют польза!!!! ;) / О/ спасибо большое *успешной ночи всем читателям моего твиттаркера =********Я знаю сколько времени вы просижите возле моей кровати )) У нас тут'

In [ ]:
text = "Мама"
inpt = tok.encode(text, return_tensors="pt")


In [ ]:
import random
n = float(random.randrange(0, 50))

In [ ]:
out = model.generate(inpt, max_length=200, repetition_penalty=3.0, do_sample=True, top_k=10, top_p=0.95, temperature=n)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
tok.decode(out[0])

'Мама. я не виновата, но у меня на лице выражение: «Ты мне в глаза смотри!»\nИ тут из класса появляется девочка с огромным пакетом :*Я хочу быть красивой:) — Нууу..не совсем...я тоже) /      - АХАХ! Ооооу!!!!!!!   да уж)) ааа ну так-да))))))) ты бы видела мое довольное лицо от уха до носа!)У кого то свадьба,кто нибудь поздравлял? ))://./5169 Я тебя понимаю. Ты сама себе удивляешься что твориться вокруг? )Сходила за билетами и теперь иду гулять к брату домой ;)Азарова как обычно спит рядом со столом;)"Продаю 2 тонны марихуанны..." ))) это ж сколько человек было сегодня!! :)"Начинаем серию ремизов по игре Хиддинки"! С нетерпением жду серии хита 2014 г…Оказывается "вторая молодость": когда девушка говорит про свою грудь он вспоминает детство'

In [ ]:
from google.colab import files
files.download('/content/models/essays/checkpoint-3500/pytorch_model.bin')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>